In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import pygame
import session_info

In [2]:
class RussellsGridEnv(gym.Env):
    metadata = {"render_modes": ["human", "ansi", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None):
        super(RussellsGridEnv, self).__init__()

        # Define the grid size
        self.height = 3
        self.width = 4

        # Define the action space
        self.action_space = spaces.Discrete(4)  # Up, Right, Down, Left

        # Define the observation space
        self.observation_space = spaces.Discrete(self.height * self.width)

        # Define the grid
        self.grid = np.zeros((self.height, self.width))
        self.grid[0, 3] = 1  # Green (terminal state)
        self.grid[1, 3] = 2  # Red (terminal state)
        self.grid[1, 1] = 3  # Black (impossible state)

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        # For rendering
        self.window = None
        self.clock = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = 8  # Start at (0, 0)
        
        if self.render_mode == "human":
            self._render_frame()
        return self.state, {}

    def step(self, action):
        row, col = divmod(self.state, self.width)

        # Determine movement
        if np.random.random() < 0.8:  # 80% chance of intended movement
            if action == 0:  # Up
                row = max(0, row - 1)
            elif action == 1:  # Right
                col = min(self.width - 1, col + 1)
            elif action == 2:  # Down
                row = min(self.height - 1, row + 1)
            elif action == 3:  # Left
                col = max(0, col - 1)
        else:  # 20% chance of random adjacent movement
            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            dr, dc = directions[np.random.randint(4)]
            row = max(0, min(self.height - 1, row + dr))
            col = max(0, min(self.width - 1, col + dc))

        # Check if new state is valid
        if self.grid[row, col] != 3:  # Not the black cell
            self.state = row * self.width + col

        # Check for terminal states
        done = self.grid[row, col] in [1, 2]
        reward = 1 if self.grid[row, col] == 1 else (-1 if self.grid[row, col] == 2 else -0.04)

        if self.render_mode == "human":
            self._render_frame()

        return self.state, reward, done, False, {}

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()
        if self.render_mode == "ansi":
            return self._render_frame()

    def _render_frame(self):
        if self.render_mode == "human":
            return self._render_human()
        elif self.render_mode == "ansi":
            return self._render_ansi()
        elif self.render_mode == "rgb_array":
            return self._render_rgb_array()

    def _render_human(self):
        if self.window is None:
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.width * 100, self.height * 100))
        if self.clock is None:
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.width * 100, self.height * 100))
        canvas.fill((255, 255, 255))

        pix_square_size = 100  # The size of a single grid square in pixels

        # Draw the grid
        for i in range(self.height):
            for j in range(self.width):
                if self.grid[i, j] == 1:
                    color = (0, 255, 0)  # Green
                elif self.grid[i, j] == 2:
                    color = (255, 0, 0)  # Red
                elif self.grid[i, j] == 3:
                    color = (0, 0, 0)  # Black
                else:
                    color = (200, 200, 200)  # Light gray
                pygame.draw.rect(
                    canvas,
                    color,
                    pygame.Rect(
                        pix_square_size * j,
                        pix_square_size * i,
                        pix_square_size,
                        pix_square_size,
                    ),
                )

        # Draw the agent
        agent_row, agent_col = divmod(self.state, self.width)
        pygame.draw.circle(
            canvas,
            (0, 0, 255),  # Blue
            (agent_col * pix_square_size + pix_square_size // 2, agent_row * pix_square_size + pix_square_size // 2),
            pix_square_size // 3,
        )

        # Add gridlines
        for x in range(self.width + 1):
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.height * pix_square_size),
                width=3,
            )
        for y in range(self.height + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * y),
                (self.width * pix_square_size, pix_square_size * y),
                width=3,
            )

        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.update()

        self.clock.tick(self.metadata["render_fps"])

    def _render_ansi(self):
        output = ""
        for i in range(self.height):
            for j in range(self.width):
                if i * self.width + j == self.state:
                    output += "A "
                elif self.grid[i, j] == 1:
                    output += "G "
                elif self.grid[i, j] == 2:
                    output += "R "
                elif self.grid[i, j] == 3:
                    output += "B "
                else:
                    output += ". "
            output += "\n"
            
        print(output)
        return 'null'

    def _render_rgb_array(self):
        
        canvas = np.zeros((self.height * 100, self.width * 100, 3), dtype=np.uint8)

        pix_square_size = 100  # The size of a single grid square in pixels

        # Draw the grid
        for i in range(self.height):
            for j in range(self.width):
                if self.grid[i, j] == 1:
                    color = [0, 255, 0]  # Green
                elif self.grid[i, j] == 2:
                    color = [255, 0, 0]  # Red
                elif self.grid[i, j] == 3:
                    color = [0, 0, 0]  # Black
                else:
                    color = [200, 200, 200]  # Light gray
                canvas[i*pix_square_size:(i+1)*pix_square_size, j*pix_square_size:(j+1)*pix_square_size] = color

        # Draw the agent
        agent_row, agent_col = divmod(self.state, self.width)
        rr, cc = np.ogrid[
            (agent_row * pix_square_size + pix_square_size // 2 - pix_square_size // 3):
            (agent_row * pix_square_size + pix_square_size // 2 + pix_square_size // 3),
            (agent_col * pix_square_size + pix_square_size // 2 - pix_square_size // 3):
            (agent_col * pix_square_size + pix_square_size // 2 + pix_square_size // 3)
        ]
        canvas[rr, cc] = [0, 0, 255]  # Blue

        return canvas

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

# Register the environment
gym.register(
    id='RussellsGrid-v0',
    entry_point='__main__:RussellsGridEnv',
)

In [3]:
env = gym.make('RussellsGrid-v0', render_mode = 'ansi')
env.reset()
env.render()

done = False
while not done:
        action = env.action_space.sample() # Take a random action
        state, reward, done, truncated, info = env.step(action)
        env.render()
        print(f"State: {state}, Reward: {reward}, Done: {done}")

. . . G 
. B . R 
A . . . 

. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: False
. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
. . A . 

State: 10, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
. . A . 

State: 10, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: False
. . . G 
. B . R 
. A . . 

State: 9, Reward: -0.04, Done: False
. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: False
. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: False
. . . G 
A B . R 
. . . . 

State: 4, Reward: -0.04, Done: False
. . . G 
. B . R 
A . . . 

State: 8, Reward: -0.04, Done: F

In [4]:
print('reset', env.reset()) # initial state is 8 (0,0)

observ, _ = env.reset()

print('env.step(action)', env.step(0))

next_state, reward, done, _, _  = env.step(0)

print(env.step(0))

print('action space [0,1,2,3] UP, RIGHT, DOWN, LEFT')



reset (8, {})
env.step(action) (4, -0.04, False, False, {})
(1, -0.04, False, False, {})
action space [0,1,2,3] UP, RIGHT, DOWN, LEFT


In [5]:
print('action space', env.action_space)
print('observation space', env.observation_space)
print('spec', env.spec)
print('metadata', env.metadata)
print('random', env.np_random)

action space Discrete(4)
observation space Discrete(12)
spec EnvSpec(id='RussellsGrid-v0', entry_point='__main__:RussellsGridEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=None, order_enforce=True, disable_env_checker=False, kwargs={'render_mode': 'ansi'}, namespace=None, name='RussellsGrid', version=0, additional_wrappers=(), vector_entry_point=None)
metadata {'render_modes': ['human', 'ansi', 'rgb_array'], 'render_fps': 4}
random Generator(PCG64)


In [6]:
session_info.show(html=False)

-----
gymnasium           1.1.1
numpy               1.26.4
pygame              2.6.1
session_info        1.0.0
-----
IPython             8.28.0
jupyter_client      8.6.3
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2025-03-10 16:45
